In [1]:
import pandas as pd

/opt/conda/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


# Load dataset

You can download & load the dataset from either Huggingface or Zenodo:

* Zenodo: [https://zenodo.org/records/10516043](https://zenodo.org/records/10516043)
* Huggingface: [https://hf.co/datasets/iDRAMALab/iDRAMA-scored-2024](https://hf.co/datasets/iDRAMALab/iDRAMA-scored-2024)

Checkout notebook on how to download data here:

* Zenodo-loaded: []()
* Huggingface-loader: []()

In [2]:
# Assuming, if you have a data-file in local (downloaded from Zenodo)
pd_df = pd.read_json("scored-submissions-2020-to-2023.ndjson", lines=True)

In [3]:
pd_df

,link,type,uuid,score,title,domain,created,is_nsfw,is_admin,is_image,...,is_deleted,is_twitter,score_down,video_link,raw_content,is_moderator,post_flair_text,post_flair_class,date,author
0,,link,3s8sOKk,62,,,1577836900810,False,False,False,...,True,False,0,,,False,,,2020-01-01,
1,https://www.lifezette.com/2019/12/joe-biden-ac...,link,3s8sOKl,41,Peak Projective Gaslighting: Joe Biden Makes T...,www.lifezette.com,1577837069302,False,False,False,...,False,False,0,,,False,,,2020-01-01,001e913d1f9cd56f3360d1a38bd2b685
2,,link,3s8sOKm,1,,,1577837522294,False,False,False,...,True,False,0,,,False,&#128002; Bullshit &#128169;,bullshit,2020-01-01,
3,https://i.imgur.com/pol5G6j.png,link,3s8sOKn,36,Kim Jong Un Hair Check,i.imgur.com,1577837559415,False,False,True,...,False,False,0,,,False,&#128169; SHITPOST &#128169;,shitpost,2020-01-01,001e913d1f9cd56f3360d1a38bd2b685
4,https://www.breitbart.com/national-security/20...,link,3s8sOOm,88,Iranian Militia Leader Leading Iraq U.S. Embas...,www.breitbart.com,1577837817679,False,False,False,...,False,False,0,,,False,,,2020-01-01,aae2e2e8c5abcae2734a6bebd8eb6ab9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6293975,https://www.infowars.com/posts/transgender-spa...,link,17s5RbwGu8,4,🚨🚨🚨🚨BREAKING: TRANSGENDER SPACE FORCE “OFFICER...,www.infowars.com,1704066938342,False,False,False,...,False,False,0,,,False,,,2023-12-31,aec9f443fca67d30571428d8d09892a4
6293976,,text,17s5RbwGu9,4,If everyone would have just washed their hands...,,1704066953914,False,False,False,...,False,False,0,,https://search.brave.com/search?q=charles+shul...,False,💉 GET BOOSTED 💉,peace,2023-12-31,7c0ee7bba12d1c190e21f5ea6b234210
6293977,,link,17s5RbwGy8,1,,,1704067038578,False,False,False,...,True,False,0,,,False,,,2023-12-31,
6293978,https://media.scored.co/post/zVIzvnBY9RxS.jpeg,link,17s5RbwGy9,2,I wonder who they will vote for...,media.scored.co,1704067095887,False,False,True,...,False,False,0,,,False,,,2023-12-31,a361d9ced8fe352b6520dce468eb267f


### Analyze top-N authors with highest submissions

In [12]:
# Filter out all records without authors information
pd_df = pd_df[pd_df.author != ""]

# Value_counts
author_freq_df = pd_df.author.value_counts().reset_index(False)

In [13]:
author_freq_df

,author,count
0,a7e2f6499a2a01b2bf34761cad2be150,43033
1,b95d95aede10b5f9d0eaf7c7aaded363,42402
2,7781a3e83e41ac6dc25560dc0745bcd0,39886
3,d3183da08f6908a1b1371bf900e9b046,33598
4,8d366f5f8186dc841261b769828866d7,32912
...,...,...
131686,25c934bd3a36fa653f1efd1e24582f00,1
131687,276c813e59d5ae9610cf73c778acf0c2,1
131688,cf495d8e9322a5f8f163fb49d1e90ed0,1
131689,2a6be1f4bd9341957501bff1ec32456a,1


# Handling null or empty records

- If field (column) doesn't have a value, the fields are left with an empty value.
  - For instance, in the case of post deletion as a moderation measure, `title` of submission can have no value.
  - We do not explicitly mark value as "Null" for any of the column in our dataset except `embedding` column.
- Only, embedding column contains explicit "Null" value.

In [ ]:
# Load dataset for `comments-2020` config
dataset = load_dataset("iDRAMALab/iDRAMA-scored-2024", name="comments-2020")
pd_df = dataset["train"].to_pandas()

# Remove all empty records based on empty `title` column
pd_df = pdf_df[pd_df.raw_content != ""]

# Remove all records which do not have `author` information
pd_df = pdf_df[pd_df.author != ""]

**Why we have no embeddings for some records?**

    - One, the content for particular post is deleted before our crawler collected it.
    - Second, we parse each post and remove URLs from the posts before generating embedding. So, if the post was simply a url and no content, no reason to generate embedding for empty string.

In [ ]:
# Remove all records which do not have generated embeddings
pd_df = pdf_df[~pd_df.embedding.isna()]